In [1]:

cd ../../../../

/Users/mykhailoslukvin/repo/dv-data-pipeline


/Users/mykhailoslukvin/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [24]:
import asyncio
import io
from io import BytesIO
import json
import os
from collections import defaultdict
from pprint import pprint as print

import aiohttp
import country_converter as coco
import pandas as pd
import requests
from tqdm.asyncio import tqdm_asyncio

from dfpp.storage import StorageManager
from dfpp.transformation.column_name_template import SexEnum

cc = coco.CountryConverter()

MAX_CONCURRENCY = 5

In [25]:
async with StorageManager() as storage_manager:
    path = os.path.join(storage_manager.utilities_path, "sep5_country_lookup.xlsx")
    data = await storage_manager.read_blob(path=path)
    df_country_map = pd.read_excel(BytesIO(data), sheet_name="country_lookup")
    df_region_map = pd.read_excel(BytesIO(data), sheet_name="region_lookup")

df_country_map = df_country_map[df_country_map["Numeric code"].notna()]
df_country_map["Numeric code"] = df_country_map["Numeric code"].astype(int)

df_region_map = df_region_map[df_region_map["Numeric code"].notna()]
df_region_map["Numeric code"] = df_region_map["Numeric code"].astype(int)

iso_3_region = dict(df_region_map[["Numeric code", "Alpha-3 code"]].values)
iso_3_country = dict(df_country_map[["Numeric code", "iso3"]].values)

iso_3_country.update(iso_3_region)

iso_3_map = iso_3_country

In [26]:
sex_remap = {
    "BOTHSEX": SexEnum.BOTH.value,
    "MALE": SexEnum.MALE.value,
    "FEMALE": SexEnum.FEMALE.value,
}

age_remap = {"ALLAGE", "all"}

In [4]:
def get_indicators():
    url = "https://unstats.un.org/sdgapi/v1/sdg/CompareTrends/GetDisaggregatedGlobalAndRegional"

    headers = {"Accept": "application/json"}

    response = requests.post(url, headers=headers)
    response.raise_for_status()

    data = response.json()
    return data

In [5]:
indicators = get_indicators()

In [27]:
series_codes = set([i["seriesCode"] for i in indicators])

In [28]:
async def fetch_data(session, series_id, semaphore):
    url = f"https://unstats.un.org/sdgapi/v1/sdg/Series/Data"
    params = {"seriesCode": series_id, "page": 1, "pageSize": 10000000}
    headers = {"Accept": "application/json"}

    async with semaphore:
        async with session.get(url, headers=headers, params=params) as response:
            response.raise_for_status()
            return await response.json()


async def process_series(series_id, session, semaphore):
    year_data = await fetch_data(session, series_id, semaphore)

    dimensions = [d["id"] for d in year_data["dimensions"]]
    size = year_data["totalElements"]
    return series_id, year_data["data"], dimensions, size


async def get_series_data_and_dimensions(
    series_codes, max_concurrent_requests=MAX_CONCURRENCY
):
    semaphore = asyncio.Semaphore(max_concurrent_requests)
    series_data_map = {}
    series_map = defaultdict(dict)

    async with aiohttp.ClientSession() as session:
        tasks = []
        for row in series_codes:
            series_id = row
            tasks.append(process_series(series_id, session, semaphore))

        results = await asyncio.gather(*tasks)

        for series_id, data, dimensions, size in results:
            series_data_map[series_id] = data
            series_map[series_id]["dimensions"] = dimensions
            series_map[series_id]["totalElements"] = size

    return series_data_map, series_map

In [20]:
for series_id in tqdm(series_codes):
    series_data_map, dimension_map = await get_series_data_and_dimensions([series_id])
    df_source = pd.DataFrame(series_data_map[series_id])
    df = df_source.copy()

    assert df.shape[0] > 0, "DataFrame is empty"

    assert (
        df.series.value_counts(dropna=False).shape[0] == 1
    ), "Multiple series values found"

    print(f"Series description: {df.seriesDescription.iloc[0]}")

    assert (
        df.shape[0] == dimension_map[series_id]["totalElements"]
    ), "Shape mismatch with expected dimensions"

    df_dimensions = pd.json_normalize(df["dimensions"])
    df_attributes = pd.json_normalize(df["attributes"])

    df = pd.concat([df.drop(columns=["dimensions"]), df_dimensions], axis=1)
    df = pd.concat([df.drop(columns=["attributes"]), df_attributes], axis=1)

    df.rename(
        columns={
            "geoAreaCode": "alpha_3_code",
            "geoAreaName": "country_or_area",
            "timePeriodStart": "year",
        },
        inplace=True,
    )

    original_dimension_columns = df_dimensions.columns
    dimension_columns = original_dimension_columns.str.lower().str.replace(
        "\s", "_", regex=True
    )
    dimension_column_rename_map = dict(
        zip(original_dimension_columns, dimension_columns)
    )

    df.rename(columns=dimension_column_rename_map, inplace=True)

    df_selection = df.copy()[
        ["alpha_3_code", "country_or_area", "year"] + dimension_columns.tolist()
    ]

    if "age" in df_selection.columns:
        df_selection["age"] = df_selection["age"].replace(age_remap)

    if "sex" in df_selection.columns:
        df_selection["sex"] = df_selection["sex"].replace(sex_remap)

    df_selection["alpha_3_code"] = df_selection["alpha_3_code"].astype(int)
    df_selection["alpha_3_code"] = df_selection["alpha_3_code"].replace(iso_3_map)

    with io.BytesIO() as output_buffer:
        df_selection.to_excel(output_buffer, index=False, engine="openpyxl")
        output_buffer.seek(0)

        async with StorageManager() as storage_manager:
            path_to_save = os.path.join(
                storage_manager.test_path, "unstats_un_org", f"{series_id}.xlsx"
            )
            blob_client = storage_manager.container_client.get_blob_client(
                blob=path_to_save
            )

            await blob_client.upload_blob(
                data=output_buffer.getvalue(),
                overwrite=True,
            )

  0%|          | 0/426 [00:00<?, ?it/s]

('Series description: Alcohol consumption per capita (aged 15 years and older) '
 'within a calendar year (litres of pure alcohol)')


/var/folders/j2/l11h93nn4s542l1h1yzw34980000gn/T/ipykernel_5610/1393530018.py:48: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df_selection["age"] = df_selection["age"].replace(age_remap)
  0%|          | 1/426 [00:18<2:12:52, 18.76s/it]

'Series description: Freight volume, by mode of transport (tonne kilometres)'


  0%|          | 2/426 [00:28<1:35:35, 13.53s/it]

'Series description: Annual growth rate of real GDP per capita (%)'


  1%|          | 3/426 [00:58<2:17:46, 19.54s/it]


CancelledError: 

In [23]:
async def process_series_id(series_id, age_remap, sex_remap, iso_3_map, storage_manager, semaphore):
    async with semaphore:
        series_data_map, dimension_map = await get_series_data_and_dimensions([series_id])
        df_source = pd.DataFrame(series_data_map[series_id])
        df = df_source.copy()

        assert df.shape[0] > 0, "DataFrame is empty"
        assert df.series.value_counts(dropna=False).shape[0] == 1, "Multiple series values found"

        print(f"Series description: {df.seriesDescription.iloc[0]}")

        assert df.shape[0] == dimension_map[series_id]["totalElements"], "Shape mismatch with expected dimensions"

        df_dimensions = pd.json_normalize(df["dimensions"])
        df_attributes = pd.json_normalize(df["attributes"])

        df = pd.concat([df.drop(columns=["dimensions"]), df_dimensions], axis=1)
        df = pd.concat([df.drop(columns=["attributes"]), df_attributes], axis=1)

        df.rename(
            columns={
                "geoAreaCode": "alpha_3_code",
                "geoAreaName": "country_or_area",
                "timePeriodStart": "year",
            },
            inplace=True,
        )

        original_dimension_columns = df_dimensions.columns
        dimension_columns = original_dimension_columns.str.lower().str.replace("\s", "_", regex=True)
        dimension_column_rename_map = dict(zip(original_dimension_columns, dimension_columns))

        df.rename(columns=dimension_column_rename_map, inplace=True)

        df_selection = df.copy()[["alpha_3_code", "country_or_area", "year"] + dimension_columns.tolist()]

        if "age" in df_selection.columns:
            df_selection["age"] = df_selection["age"].replace(age_remap)

        if "sex" in df_selection.columns:
            df_selection["sex"] = df_selection["sex"].replace(sex_remap)

        df_selection["alpha_3_code"] = df_selection["alpha_3_code"].astype(int)
        df_selection["alpha_3_code"] = df_selection["alpha_3_code"].replace(iso_3_map)

        with io.BytesIO() as output_buffer:
            df_selection.to_excel(output_buffer, index=False, engine="openpyxl")
            output_buffer.seek(0)

            path_to_save = os.path.join(storage_manager.test_path, "unstats_un_org", f"{series_id}.xlsx")
            blob_client = storage_manager.container_client.get_blob_client(blob=path_to_save)

            await blob_client.upload_blob(data=output_buffer.getvalue(), overwrite=True)


async def process_all_series(series_codes, age_remap, sex_remap, iso_3_map, max_concurrent_tasks=MAX_CONCURRENCY):
   
    semaphore = asyncio.Semaphore(max_concurrent_tasks)

    async with StorageManager() as storage_manager:
        tasks = [
            process_series_id(series_id, age_remap, sex_remap, iso_3_map, storage_manager, semaphore)
            for series_id in series_codes
        ]
       
        for task in tqdm_asyncio.as_completed(tasks):
            await task

In [ ]:
await process_all_series(series_codes, age_remap, sex_remap, iso_3_map, max_concurrent_tasks=MAX_CONCURRENCY)

  0%|          | 0/426 [00:12<?, ?it/s]


CancelledError: 

('Series description: Proportion of women who make their own informed '
 'decisions regarding contraceptive use (% of women aged 15-49 years)')


/var/folders/j2/l11h93nn4s542l1h1yzw34980000gn/T/ipykernel_5610/2778916810.py:38: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df_selection["age"] = df_selection["age"].replace(age_remap)


('Series description: Extent to which countries have laws and regulations that '
 'guarantee full and equal access to women and men aged 15 years and older to '
 'sexual and reproductive health care, information and education (%)')
('Series description: Degree of integrated water resources management '
 'implementation, management instruments (%)')
('Series description: Beach litter originating from national land-based '
 'sources that ends in the beach (%)')
('Series description: Bribery incidence (% of firms experiencing at least one '
 'bribe payment request)')
('Series description: Proportion of individuals who own a mobile telephone, by '
 'sex (%)')
('Series description: Alcohol consumption per capita (aged 15 years and older) '
 'within a calendar year (litres of pure alcohol)')


/var/folders/j2/l11h93nn4s542l1h1yzw34980000gn/T/ipykernel_5610/2778916810.py:38: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df_selection["age"] = df_selection["age"].replace(age_remap)


('Series description: Number of countries with national adaptation plans '
 '(Number)')
('Series description: Gross receipts by developing countries of official '
 'sustainable development grants (millions of United States dollars)')
('Series description: Degree of application of a '
 'legal/regulatory/policy/institutional framework which recognizes and '
 'protects access rights for small-scale fisheries (level of implementation: 1 '
 'lowest to 5 highest)')
('Series description: Total official flows for infrastructure, by recipient '
 'countries (millions of constant 2022 United States dollars)')
